In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import muon as mu
from muon import MuData
from sklearn.preprocessing import Binarizer
import h5py

In [11]:
hg38.columns

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'attribute', 'gene_id', 'gene_type', 'gene_name', 'level',
       'tag', 'transcript_id', 'transcript_type', 'transcript_name',
       'exon_number', 'exon_id', 'transcript_support_level',
       'havana_transcript', 'hgnc_id', 'havana_gene', 'ont', 'protein_id',
       'ccdsid', 'artif_dupl'],
      dtype='object')

In [57]:
import muon as mu
import scanpy as sc
import pandas as pd

# 1. 加载h5mu文件
data = mu.read_h5mu('/stor/lep/diffusion/multiome/openproblem_filtered.h5mu')

# 2. 加载基因和peak位置信息
hg38_file = "hg38/gencode_hg38.csv"  # 替换为你的gencode_hg38.csv文件路径
hg38 = pd.read_csv(hg38_file)

# 3. 筛选基因：使用HVG（高变基因）
# 假设RNA数据存储在data['rna']
sc.pp.normalize_total(data['rna'], target_sum=1e4)
sc.pp.log1p(data['rna'])

sc.pp.highly_variable_genes(data['rna'], n_top_genes=2500)  # 可以调整HVG数量
hvg_genes = data['rna'].var[data['rna'].var['highly_variable']].index.tolist()

# 4. 筛选peak：根据开放比例（例如选择开放比例>0.1的peak）
# 假设ATAC数据存储在data['atac']，且开放比例存储在obs中某列，例如"open_ratio"
open_ratio_threshold = 0.15
data['atac'].var['open_ratio'] = data['atac'].X.toarray().sum(0)/data['atac'].shape[0]
open_peaks = data['atac'].var[data['atac'].var['open_ratio'] > open_ratio_threshold].index.tolist()

# 5. 读取目标基因列表
target_genes = ["CD3E", "CD4", "NFKB2", "ZAP70"]  # 替换为你的目标基因列表

# 6. 补全目标基因上下游1M长度的peaks
def get_gene_region(gene_name, hg38_df, upstream=1e6, downstream=1e6):
    gene_info = hg38_df[hg38_df['gene_name'] == gene_name]
    if gene_info.empty:
        return None
    chrom = gene_info.iloc[0]['seqname']
    start = max(0, gene_info.iloc[0]['start'] - upstream)
    end = gene_info.iloc[0]['end'] + downstream
    return chrom, start, end

def find_peaks_in_region(chrom, start, end, peaks):
    region_peaks = []
    for peak in peaks:
        peak_chrom, peak_range = peak.split('-')[0], '-'.join(peak.split('-')[1:])
        peak_start, peak_end = map(int, peak_range.split('-'))
        if peak_chrom == chrom and peak_start < end and peak_end > start:
            region_peaks.append(peak)
    return region_peaks

# 找到所有补全的peaks
gene_regions = []
for gene in target_genes:
    region = get_gene_region(gene, hg38)
    if region:
        chrom, start, end = region
        gene_regions.extend(find_peaks_in_region(chrom, start, end, open_peaks))

# 确保目标基因补全的peaks去重
final_peaks = list(set(open_peaks + gene_regions))
final_genes = list(set(hvg_genes + target_genes))

# 7. 更新筛选后的基因和peaks
rna = data['rna'][:, final_genes]
atac = data['atac'][:, final_peaks]
mdata = mu.MuData({'rna':rna,'atac':atac})
# 8. 保存处理后的文件
output_file = "/stor/lep/diffusion/multiome/openproblem_filtered4perturb.h5mu"  # 替换为你的输出路径
mdata.write(output_file)

print(f"筛选后的数据已保存到 {output_file}")


/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/tmp/ipykernel_1008385/1369190616.py:10: DtypeWarning: Columns (22,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  hg38 = pd.read_csv(hg38_file)
/home/lep/miniconda3/envs/mm-dif

筛选后的数据已保存到 /stor/lep/diffusion/multiome/openproblem_filtered4perturb.h5mu


In [58]:
mdata

MuData object with n_obs × n_vars = 69249 × 8519
  var:	'feature_types', 'gene_id', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
  2 modalities
    rna:	69249 x 2503
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'GEX_X_umap1', 'GEX_X_umap2', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
      var:	'feature_types', 'gene_id', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism', 'log1p', 'hvg'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'
    atac:	69249 x 6016
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'ATAC_umap1', 'ATAC_umap2', 'n_genes_by_counts', 'total_counts'
      var:	'feature_types', 'gene_id', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'open_ratio'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'

In [3]:
dataset_path = '/stor/lep/diffusion/multiome/openproblem_filtered.h5mu'
mdata = mu.read_h5mu(dataset_path)
real_cell2 = mdata['atac'][mdata['atac'].obs['cell_type']=='CD4+ T naive'].X.toarray().mean(0)

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


In [4]:
bedgraph_data = []
for region, score in zip(mdata['atac'].var_names.values, real_cell2):
    chrom, start_end = region.split('-')[0], region.split('-')[1:]
    if not chrom.startswith('chr'):
        continue
    start, end = int(start_end[0]), int(start_end[1])
    bedgraph_data.append([chrom, start, end, score])

bedgraph_df = pd.DataFrame(bedgraph_data, columns=["chrom", "start", "end", "value"])
bedgraph_df.to_csv("bigwig_atac/CD4_naive.bedgraph", sep="\t", header=False, index=False)

print("Bedgraph file saved!")
# bedGraphToBigWig Transitional_B.bedgraph hg38.chrom.sizes Transitional_B.bw

Bedgraph file saved!


In [2]:
mdata1 = mu.read_h5mu('/stor/lep/data/BABEL/train.h5mu')
mdata1

ValueError: The file is not an HDF5 file

In [14]:
mdata2 = mu.read_h5mu('/stor/lep/data/BABEL/valid.h5mu')
mdata2

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 2901 × 258758
  var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
  2 modalities
    rna:	2901 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'gene_ids', 'feature_types', 'genome', 'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'
    atac:	2901 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'

In [15]:
mdata3 = mu.read_h5mu('/stor/lep/data/BABEL/test.h5mu')
mdata3

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 2004 × 258758
  var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
  2 modalities
    rna:	2004 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'gene_ids', 'feature_types', 'genome', 'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'
    atac:	2004 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'

In [16]:
mdata4 = mu.read_h5mu('/stor/lep/data/BABEL/all.h5mu')
mdata4

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 31264 × 258758
  2 modalities
    rna:	31264 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain', 'split'
      layers:	'row_counts'
    atac:	31264 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain', 'split'
      layers:	'row_counts'

In [9]:
mdata = mu.read_h5mu('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_train.h5mu')

# rna = mdata['rna']
# atac = mdata['atac']

# rna.write_h5ad('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_train_rna.h5ad')
# atac.write_h5ad('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_train_atac.h5ad')
mdata

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 8000 × 65690
  2 modalities
    rna:	8000 x 25604
      obs:	'cell_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
      var:	'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
      uns:	'cell_type_colors', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'X_counts', 'count'
      obsp:	'connectivities', 'distances'
    atac:	8000 x 40086
      obs:	'cell_type'
      uns:	'cell_type_colors', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'X_counts', 'cpm_norm', 'tf-idf'
      obsp:	'connectivities', 'distances'

In [10]:
mdata1 = mu.read_h5mu('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_train.h5mu')
mdata2 = mu.read_h5mu('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_test.h5mu')

rna = ad.concat([mdata1['rna'],mdata2['rna']])
atac = ad.concat([mdata1['atac'],mdata2['atac']])
mdata = MuData({'rna':rna,'atac':atac})
mdata

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of alway

MuData object with n_obs × n_vars = 10000 × 65690
  2 modalities
    rna:	10000 x 25604
      obs:	'cell_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
      obsm:	'X_pca', 'X_umap'
      layers:	'X_counts', 'count'
    atac:	10000 x 40086
      obs:	'cell_type'
      obsm:	'X_pca', 'X_umap'
      layers:	'X_counts', 'cpm_norm', 'tf-idf'

In [12]:
mdata.write_h5mu('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_all.h5mu')

In [63]:
adata1 = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/valid_rna.h5ad')
adata1.X = adata1.layers['row_counts']
del adata1.layers['row_counts']

adata2 = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/valid_atac.h5ad')
adata2.X = adata2.layers['row_counts']
del adata2.layers['row_counts']

adata = ad.concat((adata1,adata2),axis=1)
adata.var['feature_types'] = 
adata

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/merge.py:942: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


AnnData object with n_obs × n_vars = 2901 × 258758
    var: 'chrom', 'n_counts', 'log1p_counts', 'n_cells'

In [60]:
adata1 = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/valid_atac.h5ad')
adata1.X = adata1.layers['row_counts']
del adata1.layers['row_counts']

adata2 = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/valid_rna.h5ad')
adata2.X = adata2.layers['row_counts']
del adata2.layers['row_counts']

adata = ad.concat((adata1,adata2))

# 获取数据
X = adata.X
barcode = adata.obs.index.values
gene = adata.var.index.values

# 创建一个新的HDF5文件
with h5py.File('/stor/lep/workspace/multi_diffusion/my_model/valid_atac.h5', 'w') as f:
    # 创建一个组用于存放数据
    group = f.create_group('matrix')
    
    # 存储基因名称
    dset_genes = group.create_dataset('genes', (len(gene),), dtype=h5py.string_dtype())
    dset_genes[:] = gene
    
    # 存储条形码
    dset_barcodes = group.create_dataset('barcodes', (len(barcode),), dtype=h5py.string_dtype())
    dset_barcodes[:] = barcode
    
    # 存储数据矩阵
    # 10x格式通常使用CSR格式存储稀疏矩阵
    if isinstance(X, np.ndarray):
        # 如果矩阵不是稀疏的，将其转换为CSR格式
        from scipy.sparse import csr_matrix
        X = csr_matrix(X)

    # 提取CSR格式的数据
    data = X.data
    indices = X.indices
    indptr = X.indptr
    shape = X.shape
    
    # 存储CSR格式的数据
    group.create_dataset('data', data=data)
    group.create_dataset('indices', data=indices)
    group.create_dataset('indptr', data=indptr)
    group.attrs['shape'] = shape


    # 存储基因注释
    # gene_attrs = adata.var.to_dict('list')
    for attr_name, attr_values in gene_attrs.items():
        # 创建一个数据集来存储属性值
        dset_attr = group.create_dataset(f'genes/{attr_name}', (len(attr_values),), dtype=h5py.string_dtype())
        dset_attr[:] = attr_values
    
    # 存储细胞注释
    # barcode_attrs = adata.obs.to_dict('list')
    for attr_name, attr_values in barcode_attrs.items():
        # 创建一个数据集来存储属性值
        dset_attr = group.create_dataset(f'barcodes/{attr_name}', (len(attr_values),), dtype=h5py.string_dtype())
        dset_attr[:] = attr_values

TypeError: Incompatible object (Dataset) already exists

In [64]:
adata2 = sc.read_10x_h5('/stor/lep/data/BABEL/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5')
adata2

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 11909 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [65]:
adata2.var['feature_types']

MIR1302-2HG    Gene Expression
FAM138A        Gene Expression
OR4F5          Gene Expression
AL627309.1     Gene Expression
AL627309.3     Gene Expression
                    ...       
AC141272.1     Gene Expression
AC023491.2     Gene Expression
AC007325.1     Gene Expression
AC007325.4     Gene Expression
AC007325.2     Gene Expression
Name: feature_types, Length: 36601, dtype: object

In [68]:
adatatt = sc.read_10x_h5('/stor/lep/data/BABEL/testset/GSM5085810_GM12878_rep1_filtered_feature_bc_matrix.h5')
adatatt

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 3509 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

read h5 data

In [44]:
# adata111 = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/scDesign3/output/sim_open_RNA.h5')
import h5py

# 打开HDF5文件
with h5py.File('/stor/lep/workspace/multi_diffusion/scDesign3/output/sim_open_RNA.h5', 'r') as hdf:
# with h5py.File('./output/sim_open_RNA.h5', 'r') as hdf:
    # 读取矩阵数据
    matrix = np.array(hdf['group/matrix'])
    
    # 读取行名和列名
    rownames = [rn.decode('utf-8') for rn in hdf['group/rownames'][:]]
    colnames = [cn.decode('utf-8') for cn in hdf['group/colnames'][:]]

# 打印矩阵、行名和列名
print("Matrix:\n", matrix)
print("Row names:", rownames)
print("Column names:", colnames)


Matrix:
 [[0.00000000e+00 8.97043549e-02 1.57706854e-01 ... 0.00000000e+00
  0.00000000e+00 3.82409853e-02]
 [0.00000000e+00 9.45176623e-04 4.79947154e-03 ... 9.04866182e-02
  0.00000000e+00 1.72568692e-03]
 [0.00000000e+00 4.59759595e-02 2.66933778e-01 ... 0.00000000e+00
  0.00000000e+00 3.74338335e-02]
 ...
 [0.00000000e+00 1.27109900e-02 1.26571916e-04 ... 7.96263632e-04
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.41640786e-01 1.36947875e-02 ... 0.00000000e+00
  0.00000000e+00 7.87642766e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.84063396e-02
  0.00000000e+00 0.00000000e+00]]
Row names: ['AL627309.5', 'LINC01409', 'LINC01128', 'NOC2L', 'KLHL17', 'ISG15', 'C1orf159', 'SDF4', 'B3GALT6', 'UBE2J2', 'ACAP3', 'INTS11', 'DVL1', 'AURKAIP1', 'CCNL2', 'MRPL20', 'ATAD3B', 'ATAD3A', 'SSU72', 'MIB2', 'MMP23B', 'CDK11B', 'SLC35E2B', 'CDK11A', 'SLC35E2A', 'NADK', 'GNB1', 'PRKCZ', 'FAAP20', 'SKI', 'MORN1', 'RER1', 'PEX10', 'PLCH2', 'PANK4', 'AL139246.5', 'TNFRSF14-AS1', 'TNFR

generate h5mu

In [6]:
# mdata = mu.read_10x_h5('/stor/lep/data/pbmc10k_multiome/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5')
mdata = mu.read_h5mu('/stor/lep/diffusion/multiome/openproblem.h5mu')
# mdata = mu.read_10x_h5('/stor/lep/data/BABEL/GSM5085810_GM12878_rep2_filtered_feature_bc_matrix.h5')
mdata.var_names_make_unique()

mdata

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 69249 × 129921
  var:	'feature_types', 'gene_id'
  2 modalities
    atac:	69249 x 116490
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'ATAC_umap1', 'ATAC_umap2'
      var:	'feature_types', 'gene_id'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'
    rna:	69249 x 13431
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'GEX_X_umap1', 'GEX_X_umap2'
      var:	'feature_types', 'gene_id'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'

In [7]:
rna = mdata.mod['rna']
rna.var['mt'] = rna.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
mu.pp.filter_var(rna, 'n_cells_by_counts', lambda x: x >= 10)
mdata

MuData object with n_obs × n_vars = 69249 × 129921
  var:	'feature_types', 'gene_id'
  2 modalities
    atac:	69249 x 116490
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'ATAC_umap1', 'ATAC_umap2'
      var:	'feature_types', 'gene_id'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'
    rna:	69249 x 13431
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'GEX_X_umap1', 'GEX_X_umap2', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
      var:	'feature_types', 'gene_id', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'

In [8]:
atac = mdata.mod['atac']
sc.pp.calculate_qc_metrics(atac, percent_top=None, log1p=False, inplace=True)
mu.pp.filter_var(atac, 'n_cells_by_counts', lambda x: x >= int(atac.shape[0]*0.02))
# mu.atac.pp.binarize(mdata)
mdata

MuData object with n_obs × n_vars = 69249 × 129921
  var:	'feature_types', 'gene_id'
  2 modalities
    atac:	69249 x 36553
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'ATAC_umap1', 'ATAC_umap2', 'n_genes_by_counts', 'total_counts'
      var:	'feature_types', 'gene_id', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'
    rna:	69249 x 13431
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'GEX_X_umap1', 'GEX_X_umap2', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
      var:	'feature_types', 'gene_id', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'

In [109]:
# 使用muon计算TF-IDF
mu.atac.pp.tfidf(atac, scale_factor=1e4)

# 获取TF-IDF矩阵
tfidf_matrix = atac.X  # 这将是一个稀疏矩阵

# 创建Binarizer实例，设置阈值
binarizer = Binarizer(threshold=0.0)

# 对TF-IDF矩阵进行二值化处理
tfidf_matrix_binarized = binarizer.fit_transform(tfidf_matrix)

# 将二值化后的矩阵放回MuData对象（如果需要）
atac.X = tfidf_matrix_binarized

In [110]:
df = pd.read_csv('/stor/lep/data/pbmc10k_multiome/labels_annot.txt',sep='\t')
# df.head()

In [111]:
mdata.obs['cell_type'] = ['none']*mdata.shape[0]
rna.obs['cell_type'] = ['none']*mdata.shape[0]
atac.obs['cell_type'] = ['none']*mdata.shape[0]
for i in range(mdata.shape[0]):
    barcode = mdata.obs_names[i]
    celltype = df[df['barcode']==barcode]['celltype'].values[0] if len(df[df['barcode']==barcode]['celltype'].values)>0 else 'none'
    mdata.obs['cell_type'][i] = celltype
    rna.obs['cell_type'][i] = celltype
    atac.obs['cell_type'][i] = celltype

# mdata[mdata.obs['cell_type']!='none']

/tmp/ipykernel_2914482/667162776.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna.obs['cell_type'][i] = celltype
/tmp/ipykernel_2914482/667162776.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atac.obs['cell_type'][i] = celltype


In [112]:
atac = atac[(mdata.obs['cell_type']!='none').values]
rna = rna[(mdata.obs['cell_type']!='none').values]

In [113]:
mdata.mod['rna']=rna
mdata.mod['atac']=atac
mdata

MuData object with n_obs × n_vars = 11898 × 180488
  obs:	'cell_type'
  var:	'gene_ids', 'feature_types', 'genome', 'interval'
  2 modalities
    rna:	10032 x 22017
      obs:	'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'cell_type'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    atac:	10032 x 42870
      obs:	'n_genes_by_counts', 'total_counts', 'cell_type'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [114]:
mu.write("/stor/lep/data/pbmc10k_multiome/pbmc10k_filtered2.h5mu", mdata)
# mu.write("/stor/lep/diffusion/multiome/openproblem_filtered.h5mu", mdata)

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = 

In [119]:
mdata = mu.read_h5mu('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_train.h5mu')
mdata

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 8000 × 65690
  2 modalities
    rna:	8000 x 25604
      obs:	'cell_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
      var:	'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
      uns:	'cell_type_colors', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'X_counts', 'count'
      obsp:	'connectivities', 'distances'
    atac:	8000 x 40086
      obs:	'cell_type'
      uns:	'cell_type_colors', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'X_counts', 'cpm_norm', 'tf-idf'
      obsp:	'connectivities', 'distances'

In [120]:
mdata2 = mu.read_h5mu('/stor/lep/data/pbmc10k_multiome/cfgen/pbmc10k_multiome_test.h5mu')
mdata2

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 2000 × 65690
  2 modalities
    rna:	2000 x 25604
      obs:	'cell_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
      var:	'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
      uns:	'cell_type_colors', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'X_counts', 'count'
      obsp:	'connectivities', 'distances'
    atac:	2000 x 40086
      obs:	'cell_type'
      uns:	'cell_type_colors', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'X_counts', 'cpm_norm', 'tf-idf'
      obsp:	'connectivities', 'distances'

generate h5ad

In [116]:
adata = ad.concat((rna,atac),axis=1, join='inner')
adata

AnnData object with n_obs × n_vars = 10032 × 64887
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [117]:
adata.obs['cell_type'] = rna.obs['cell_type']

adata.var.feature_types[:22017] = ['GEX']*22017
adata.var.feature_types[22017:] = ['ATAC']*42870
adata

/tmp/ipykernel_2914482/2986023047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.var.feature_types[:22017] = ['GEX']*22017
/tmp/ipykernel_2914482/2986023047.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.var.feature_types[22017:] = ['ATAC']*42870


AnnData object with n_obs × n_vars = 10032 × 64887
    obs: 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [118]:
adata.write_h5ad('/stor/lep/data/pbmc10k_multiome/pbmc10k_filtered2.h5ad')
# rna.write_h5ad('/stor/lep/data/pbmc10k_multiome/pbmc10k_RNA.h5ad')
# atac.write_h5ad('/stor/lep/data/pbmc10k_multiome/pbmc10k_ATAC.h5ad')


# adata.write_h5ad('/stor/lep/diffusion/multiome/GSE194122_openproblems_neurips2021_multiome_BMMC_processed_filtered.h5ad')
# rna.write_h5ad('/stor/lep/diffusion/multiome/openproblems_RNA_new.h5ad')
# atac.write_h5ad('/stor/lep/diffusion/multiome/openproblems_ATAC_new.h5ad')

In [2]:
mdata = mu.read_h5mu('/stor/lep/diffusion/multiome/openproblem_filtered.h5mu')
mdata

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 69249 × 49984
  var:	'feature_types', 'gene_id', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
  2 modalities
    atac:	69249 x 36553
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'ATAC_umap1', 'ATAC_umap2', 'n_genes_by_counts', 'total_counts'
      var:	'feature_types', 'gene_id', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'
    rna:	69249 x 13431
      obs:	'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'GEX_X_umap1', 'GEX_X_umap2', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
      var:	'feature_types', 'gene_id', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
      uns:	'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
      obsm:	'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
      layers:	'counts'

In [6]:
mdata['rna'].write_h5ad('/stor/lep/diffusion/multiome/openproblems_RNA_filterd.h5ad')
mdata['atac'].write_h5ad('/stor/lep/diffusion/multiome/openproblems_ATAC_filterd.h5ad')

generated babel dataset

In [110]:
adata_rna_train = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/train_rna.h5ad')
adata_atac_train = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/train_atac.h5ad')
adata_rna_train.X = adata_rna_train.layers['row_counts']
adata_atac_train.X = adata_atac_train.layers['row_counts']

mdata_train = mu.MuData({'rna':adata_rna_train,'atac':adata_atac_train})
# mdata_train.write_h5mu('/stor/lep/data/BABEL/train.h5mu')
mdata_train

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 28363 × 258758
  var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
  2 modalities
    rna:	28363 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'gene_ids', 'feature_types', 'genome', 'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'
    atac:	28363 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'

In [111]:
adata_rna_valid = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/valid_rna.h5ad')
adata_atac_valid = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/valid_atac.h5ad')
adata_rna_valid.X = adata_rna_valid.layers['row_counts']
adata_atac_valid.X = adata_atac_valid.layers['row_counts']

mdata_valid = mu.MuData({'rna':adata_rna_valid,'atac':adata_atac_valid})
# mdata_valid.write_h5mu('/stor/lep/data/BABEL/valid.h5mu')
mdata_valid

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 2901 × 258758
  var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
  2 modalities
    rna:	2901 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'gene_ids', 'feature_types', 'genome', 'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'
    atac:	2901 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'

In [116]:
adata_rna_test = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/test_rna.h5ad')
adata_atac_test = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/test_atac.h5ad')
adata_rna_test.X = adata_rna_test.layers['row_counts']
adata_atac_test.X = adata_atac_test.layers['row_counts']

mdata_test = mu.MuData({'rna':adata_rna_test,'atac':adata_atac_test})
mdata_test.write_h5mu('/stor/lep/data/BABEL/test.h5mu')
mdata_test

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 2004 × 258758
  var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
  2 modalities
    rna:	2004 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'gene_ids', 'feature_types', 'genome', 'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'
    atac:	2004 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden', 'louvain'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'

In [106]:
adata_rna_test = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/12878_rna.h5ad')
adata_rna_test.X = adata_rna_test.layers['row_counts']
adata_rna_test.obs['leiden'] = '33'
adata_rna_test.obs['leiden'] = pd.Categorical(adata_rna_test.obs['leiden'])
adata_atac_test = sc.read_h5ad('/stor/lep/workspace/multi_diffusion/my_model/12878_atac.h5ad')
adata_atac_test.X = adata_atac_test.layers['row_counts']
adata_atac_test.obs['leiden'] = '33'
adata_atac_test.obs['leiden'] = pd.Categorical(adata_atac_test.obs['leiden'])

mdata_test = mu.MuData({'rna':adata_rna_test,'atac':adata_atac_test})
mdata_test.write_h5mu('/stor/lep/data/BABEL/12878.h5mu')
mdata_test

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


MuData object with n_obs × n_vars = 7095 × 258758
  var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
  2 modalities
    rna:	7095 x 34861
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'
    atac:	7095 x 223897
      obs:	'batch', 'source_file', 'n_counts', 'log1p_counts', 'n_genes', 'size_factors', 'leiden'
      var:	'chrom', 'n_counts', 'log1p_counts', 'n_cells'
      uns:	'median_counts'
      layers:	'row_counts'

In [117]:
adata_rna_all = ad.concat((adata_rna_train,adata_rna_valid, adata_rna_test),label='split')
adata_atac_all = ad.concat((adata_atac_train,adata_atac_valid, adata_atac_test),label='split')
# adata_rna_all.obs['leiden'] = pd.Categorical(adata_rna_all.obs['leiden'])
# adata_atac_all.obs['leiden'] = pd.Categorical(adata_atac_all.obs['leiden'])

mdata_all = mu.MuData({'rna':adata_rna_all,'atac':adata_atac_all})
mdata_all.write_h5mu('/stor/lep/data/BABEL/all_wtest.h5mu')

/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/anndata/_core/merge.py:942: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col
/home/lep/miniconda3/envs/mm-diffusion/lib/python3.8/site-packages/mudata/_core/mudata.py:578: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_mod.loc[:, colname] = col


In [ ]:
import muon as mu
mdata = mu.read_h5mu('path/to/h5mu/data')
print(mdata)

rna_data = mdata['rna'].X.toarray()
atac_data = mdata['atac'].X.toarray()

# the meta information can be find in mdata['rna'].obs
other_meta_info = mdata['rna'].obs['the/meta/info/you/want']